# Chapter 6. 파이프라인
이전 단원까지 배운 여러 가지 전처리 기법은 대부분의 경우 모델 튜닝, 즉 하이퍼파라미터 탐색을 할 때 공통적으로 시행하는 반복 작업~~노가다~~이 될 가능성이 매우 높습니다. 아무리 파일을 복사하고 코드를 복붙한다고 한들 사람이 하는 일이라면 언제나 실수를 할 가능성이 있습니다. 무엇보다도 **번거롭잖아요**. 그래서 이 일련의 과정을 하나의 공정처럼 체계화 및 루틴화시킴으로써 작업을 더욱 편리하게 할 수 있는 방법이 있습니다. 우리는 이 방법론을 **파이프라인**(pipeline)이라고 부르며, 이를 사용하면 여러 개의 변환 단계를 포함해 모델 학습과 새로운 데이터 예측까지 자동화할 수 있습니다. 이번 단원에서는 파이프라인을 이용해 머신러닝 개발이라는 일련의 과정을 자동화하는 연습을 해보겠습니다.  
참고로 이 단원 자체의 내용은 아주 단순하고 자체적으로는 실습이랄 것도 없지만, 이후 단원에서도 사용될 예정입니다. 또한 MLOps 과정에 있어서 가장 중요한 업무 자동화의 일환이므로 실무에서도 파이프라인을 많이 사용할 테니 꼭 사이킷런이 아니더라도 에어플로우(Airflow) 등의 툴을 사용할 때도 그 개념을 잘 알아두고 가는 것이 좋습니다.

## 6.1 들어가기 전에: 위스콘신 유방암 데이터셋
이번 단원과 7단원 교차검증에 걸쳐서는 위스콘신 유방암 데이터셋을 사용하겠습니다. 이 데이터셋에는 악성과 양성인 종양 세포 샘플 569개가 들어있습니다. 데이터셋의 첫 두 열에는 샘플의 고유 ID 번호와 진단 결과(`M`: 악성, `B`: 양성)가 들어 있습니다. 3번째부터 32번째 열까지는 세포 핵의 디지털 이미지에서 계산된 30개의 실수 값 특성이 담겨 있습니다. 이 특성을 사용해 종양이 악성인지 양성인지 예측하는 모델을 만들겠습니다.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',
                 header=None)

In [3]:
X = df.loc[:, 2:].values
y = df.loc[:, 1].values

In [4]:
le = LabelEncoder()

In [5]:
y = le.fit_transform(y)

In [6]:
le.classes_

array(['B', 'M'], dtype=object)

In [7]:
# 제대로 변환되었는지 테스트하는 더미 데이터
le.transform(['M', 'B'])

array([1, 0])

In [8]:
# 훈련 데이터셋과 테스트 데이터셋을 80:20 비율로 변환
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    stratify=y, random_state=1)

## 6.2 파이프라인으로 변환기와 추정기 연결
많은 머신러닝 알고리즘은 최적의 성능을 위해 입력 특성이 같은 스케일을 가져야 한다고 누누이 강조해왔습니다. 위스콘신 유방암 데이터셋은 다양한 스케일로 측정되었기 때문에 로지스틱 회귀 같은 분류기에 입력하기 전에 특성을 표준화할 필요가 있습니다. 여기서는 5단원에서 소개한 주성분 분석(PCA)을 통해 초기 30차원에서 좀 더 낮은 2차원 부분공간으로 데이터를 압축한다고 가정하겠습니다. ~~그 편이 시각화하기도 편하고~~  
훈련 데이터셋과 테스트 데이터셋을 각각 학습하고 변환하는 단계를 구성하는 대신 `StandardScaler`, `PCA`, `LogisticRegression` 세 객체를 하나의 파이프라인으로 연결할 수 있습니다.

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

In [10]:
# 여러 개의 변환기와 추정기를 연결하는 make_pipeline 함수
# 입력받은 객체들을 사용해 사이킷런의 Pipeline 객체를 생성해 반환
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state=1))

In [12]:
pipe_lr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])

In [13]:
y_pred = pipe_lr.predict(X_test)

In [14]:
print(f"테스트 정확도: {pipe_lr.score(X_test, y_test):.3f}")

테스트 정확도: 0.956


사이킷런의 파이프라인은 매우 유용한 래퍼(wrapper) 도구입니다. 각종 추정기나 개별 변환기를 감싼 객체라는 뜻이죠. 앞으로 나머지 과정에서 자주 사용하게 될 것입니다. `Pipeline` 객체가 동작하는 방식을 확실하게 이해하기 위해 다음 그림을 자세히 살펴보세요. 이 문단의 코드를 도식화한 것입니다.  
![그림 6-1](images/Fig6_1.png)  

> <span style='background-color:green'>Tips and Tricks</span> 파이프라인 꿀팁 & 구조 시각화<br/>
> 훈련 데이터셋을 전처리할 때 사용한 파라미터로 테스트 데이터셋을 변환해야 합니다. 비슷하게 다음 단원에서 소개할 교차 검증도 훈련 폴드에서 학습한 파라미터를 사용하여 테스트 폴드를 전처리해야 합니다. 이렇게 하려면 전처리 과정이 교차검증 반복 안으로 들어가야 합니다. 전처리 과정을 `Pipeline`으로 감싸서 교차검증 함수로 전달하면 손쉽게 구현할 수 있습니다. 자세한 내용은 7단원에서 볼 수 있습니다.  
> 한편, 사이킷런 0.23 버전부터는 추정기 객체를 주피터 노트북에서 시각화해 주는 기능이 추가되었습니다. 특히 파이프라인 구조를 시각적으로 잘 요약해주며 다음과 같이 사용할 수 있습니다.

In [16]:
from sklearn import set_config

In [17]:
set_config(display='diagram')

In [18]:
pipe_lr

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression(random_state=1))])